In [2]:
# Instalación de paquetes necesarios
!pip3 install tqdm==4.62.0
!pip3 install pandas==1.3.1
!pip3 install numpy==1.21.1

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Importación de paquetes
import pandas as pd
import numpy as np
import glob
import json
import time
from tqdm import tqdm

In [4]:
# directorio principal para todo el conjunto de datos
input_dir = 'G:/BertV1/2022-01-03'

In [5]:
# metadata - metadatos que serán empleados para obtener los documentos por id, titulo y resumen
metadata_path = f'{input_dir}/metadata.csv'
metadata = pd.read_csv(metadata_path,
                               dtype={'pubmed_id': str,'title': str,'abstract': str})
metadata.head()

c:\Users\Alexis\Desktop\SistemaQABert\Data\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (1,4,5,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [6]:
# Obtención del número de trabajos de investigación en la carpeta pdf_json
all_json_pdf = glob.glob(f'{input_dir}/document_parses/pdf_json/*.json',recursive=True)
len(all_json_pdf)

320305

In [7]:
# Obtención del número de trabajos de investigación en la carpeta pmc_json
all_json_pmc = glob.glob(f'{input_dir}/document_parses/pmc_json/*.json',recursive=True)
len(all_json_pmc)

248997

In [8]:
# Clase para extraer la identificación del artículo, el resumen y cuerpo de los trabajos de investigación.
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = "$$$".join([each['text']  for each in content.get('abstract', [])])
            self.body_text = "$$$".join([each['text']  for each in content['body_text']])
    def __repr__(self):
        return f'{self.paper_id}:\nabstract:  {self.abstract[:200]}\nbody:  {self.body_text[:200]}...'

In [9]:
# Se muestra un trabajo de investigación de la carpeta pdf_json
first_row = FileReader(all_json_pdf[0])
print(first_row)

0000028b5cc154f68b8a269f6578f21e31f62977:
abstract:  
body:  According to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. It remain...


In [10]:
# Creación de un diccionario de todos los trabajos de investigación de la carpeta pdf_json
dict_ = {'paper_id': [], 'abstract': [],'body_text': []}
t1 = time.time()
for idx, entry in tqdm(enumerate(all_json_pdf)):
    try:
        content = FileReader(entry)
        dict_['paper_id'].append(content.paper_id)
        dict_['abstract'].append(content.abstract)
        dict_['body_text'].append(content.body_text)
    except:
        print("An exception occurred")
print(time.time() - t1)

320305it [12:38, 422.17it/s]


758.8862829208374


In [11]:
# Creación de un marco de datos(dataframe) de todos los trabajos de investigación de pdf_json
df_json_pdf = pd.DataFrame(dict_, columns=['paper_id', 'abstract','body_text'])
df_json_pdf.head()

,paper_id,abstract,body_text
0,0000028b5cc154f68b8a269f6578f21e31f62977,,According to current live statistics at the ti...
1,0000b6da665726420ab8ac9246d526f2f44d5943,Objective: An at home-test for differentiating...,As a consequence of the global COVID-19 pandem...
2,0000b93c66f991236db92dc16fa6db119b27ca12,,Infections are an important cause of morbidity...
3,0000fcce604204b1b9d876dc073eb529eb5ce305,Contribución de los autores: Rocío Menéndez Co...,Se calcula que en España existen cerca de 367....
4,000122a9a774ec76fa35ec0c0f6734e7e8d0c541,Introduction and objectives: The COVID-19 outb...,Introduction and objectives: The COVID-19 outb...


In [12]:
# Creación un diccionario de todos los trabajos de investigación de pmc_json
dict_ = {'paper_id': [],'body_text': []}
t1 = time.time()
for idx, entry in tqdm(enumerate(all_json_pmc)):
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
print(time.time() - t1)

248997it [09:48, 422.80it/s]

589.0071663856506


In [13]:
# Creación de un marco de datos(dataframe) de todos los trabajos de investigación de pmc_json
df_json_pmc = pd.DataFrame(dict_, columns=['paper_id','body_text'])
df_json_pmc.head()

,paper_id,body_text
0,PMC1054884,As increasing numbers of full-length viral seq...
1,PMC1065028,The sudden emergence of severe acute respirato...
2,PMC1065064,The rate of expansion of medical knowledge is ...
3,PMC1065120,This series of articles provides regular surve...
4,PMC1065257,Programmed −1 ribosomal frameshifting (hereaft...


Muestreo y almacenamiento del conjunto de datos del curso

In [14]:
# Eliminación de las filas de los metadatos donde el texto de investigación correspondiente no existe en pdf_json
metadata.dropna(subset=['pdf_json_files'], inplace=True)

In [15]:
# Se eligen 50000 registros aleatoriamente de los metadatos
sub_metadata = metadata.sample(50000)

In [16]:
# Se filtra la tabla de trabajos de investigación tanto en pdf como en pmc según la tabla de metadatos muestreados
sub_df_json_pdf = df_json_pdf[df_json_pdf['paper_id'].isin(sub_metadata['sha'])]
sub_df_json_pmc = df_json_pmc[df_json_pmc['paper_id'].isin(sub_metadata['pmcid'])]

In [ ]:
# Almacenamiento de los datos de muestra
sub_metadata.to_pickle('metadata_sample.pickle')
sub_df_json_pdf.to_pickle('json_pdf_sample.pickle')
sub_df_json_pmc.to_pickle('json_pmc_sample.pickle')